In [1]:
import pandas as pd
import numpy as np
from surprise import NormalPredictor
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

from read_and_split_data import split_data, filter_animes_without_grade

In [2]:
def get_dataset():
    anime = pd.read_parquet("../datasets/anime.parquet")
    anime = anime[["anime_id", "type"]]
    users = pd.read_parquet("../datasets/users.parquet")
    base_df = users.merge(anime, on="anime_id", how="left")
    return base_df

base_df = get_dataset()
data = split_data(base_df)

In [3]:
kf = KFold(n_splits=5)

In [4]:
def predict_and_error(data, algo, path):
    fold = 0
    predictions_list = []
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        predictions_list.append(predictions)
        # Mean Squared Error
        accuracy.rmse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mae(predictions, verbose=True)
        # Fraction of Concordant Pairs
        accuracy.fcp(predictions, verbose=True)
        base_df = pd.DataFrame(columns=["user", "anime", "actual", "est", "details"], data=predictions)
        base_df.to_csv(f"{path}_{fold}.csv")
        base_df.to_parquet(f"{path}_{fold}.parquet")
        fold += 1
    return predictions

In [5]:
algo_normal_predictor = NormalPredictor()

In [6]:
predictions = predict_and_error(data, algo_normal_predictor, path="../predictions/normal_predictor/anime_type_tv/no_filter/normal_predictor")

RMSE: 4.6742
MSE: 21.8481
MAE:  3.7090
FCP:  0.4881
RMSE: 4.6716
MSE: 21.8235
MAE:  3.7085
FCP:  0.4865
RMSE: 4.6686
MSE: 21.7959
MAE:  3.7055
FCP:  0.4880
RMSE: 4.6655
MSE: 21.7673
MAE:  3.7036
FCP:  0.4880
RMSE: 4.6659
MSE: 21.7707
MAE:  3.7032
FCP:  0.4871


In [7]:
base_df_without_negative = filter_animes_without_grade(base_df)

In [8]:
predictions_without_negative = predict_and_error(data, algo_normal_predictor, path="../predictions/normal_predictor/anime_type_tv/with_filter_remove_negative/normal_predictor")

RMSE: 4.6707
MSE: 21.8155
MAE:  3.7051
FCP:  0.4881
RMSE: 4.6699
MSE: 21.8083
MAE:  3.7064
FCP:  0.4855
RMSE: 4.6798
MSE: 21.9005
MAE:  3.7142
FCP:  0.4873
RMSE: 4.6738
MSE: 21.8446
MAE:  3.7099
FCP:  0.4882
RMSE: 4.6727
MSE: 21.8338
MAE:  3.7084
FCP:  0.4880
